## Set up the experiment folder and move your training CSV file into the data folder within Challenge 2

In [ ]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'driver-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/porto_seguro_safe_driver_prediction_input.csv', os.path.join(training_folder, "porto_seguro_safe_driver_prediction_input.csv"))


In [ ]:
import lightgbm as lgb
"lightgbm=={}".format(lgb.__version__)

## train.py
This file defines the key functions required to train the model.  
The file can be invoked with `python train.py` for development purposes.

In [ ]:
%%writefile $training_folder/train.py
import os
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import lightgbm


def split_data(data_df):
    """Split a dataframe into training and validation datasets"""
    
    ## TODO
    
    return (train_data, valid_data)


def train_model(data, parameters):
    """Train a model with the given datasets and parameters"""
    # The object returned by split_data is a tuple.
    # Access train_data with data[0] and valid_data with data[1]
    
    ## TODO
    
    return model


def get_model_metrics(model, data):
    """Construct a dictionary of metrics for the model"""
    
    ## TODO
    
    return model_metrics


def main():
    """This method invokes the training functions for development purposes"""
    
    # Read data from a file
    data_df = pd.read_csv('porto_seguro_safe_driver_prediction_input.csv')

    # Hard code the parameters for training the model
    parameters = {
        'learning_rate': 0.02,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'sub_feature': 0.7,
        'num_leaves': 60,
        'min_data': 100,
        'min_hessian': 1,
        'verbose': 2
    }

    # Call the functions defined in this file
    ## TODO
    
    # Print the resulting metrics for the model
    ## TODO
    
if __name__ == '__main__':
    main()


## parameters.json
This file will specify the parameters used to train the model.

In [ ]:
%%writefile $training_folder/parameters.json
{
    "training":
    {
        "learning_rate": 0.02,
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "auc",
        "sub_feature": 0.7,
        "num_leaves": 60,
        "min_data": 100,
        "min_hessian": 1,
        "verbose": 0
    }
}


## driver_training.py
This file will be the entry script when running an Azure ML context.  
It calls the functions defined in train.py for data preparation and training, but reads parameters from a file, and logs output to the Azure ML context.  
The file can be invoked with `python driver_training.py` for development purposes.

In [ ]:
%%writefile $training_folder/driver_training.py
# Import libraries
import argparse
from azureml.core import Run
import joblib
import json
import os
import pandas as pd
import shutil

# Import functions from train.py
from train import split_data, train_model, get_model_metrics

# Get the output folder for the model from the '--output_folder' parameter
parser = argparse.ArgumentParser()
parser.add_argument('--output_folder', type=str, dest='output_folder', default="outputs")
args = parser.parse_args()
output_folder = args.output_folder

# Get the experiment run context
run = Run.get_context()

# load the safe driver prediction dataset
train_df = pd.read_csv('porto_seguro_safe_driver_prediction_input.csv')

# Load the parameters for training the model from the file
with open("parameters.json") as f:
    pars = json.load(f)
    parameters = pars["training"]

# Log each of the parameters to the run
for param_name, param_value in parameters.items():
    run.log(param_name, param_value)
    
# Use the functions imported from train.py to prepare data, train the model, and calculate the metrics
## TODO

# Save the trained model to the output folder
os.makedirs(output_folder, exist_ok=True)
output_path = output_folder + "/porto_seguro_safe_driver_model.pkl"
joblib.dump(value=model, filename=output_path)

run.complete()

In [ ]:
output_folder = "outputs"
os.makedirs(output_folder, exist_ok=True)
output_path = output_folder + "/porto_seguro_safe_driver_model.pkl"
print(output_path)

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace
ws = Workspace.from_config()

## Use ScriptRunConfig to run the script as an Experiment

Complete each of the steps in separate Notebook cells.

1. First, create a [compute cluster](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python)

1. [Create an environment](https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment/#in-azure-ml-sdk) and add the conda and pip packages beforing registering it into the Workspace. Use the pip, scikit-learn and lightgbm conda packages

1. [Create an experiment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py), retrieve the compute target and environment before configuring and submiting your training run using the ScriptRunConfig() object

1. Return the model metrics 

1. Complete the experiment run

In [ ]:
# Create an compute cluster
## TODO

In [ ]:
# Create an environment
## TODO

In [ ]:
# Create an experiment
## TODO

In [ ]:
# Print the resulting metrics
metrics = run.get_metrics()
for k, v in metrics.items():
        print(k, v)

In [ ]:
# Register the model
run.register_model(model_path='outputs/porto_seguro_safe_driver_model.pkl', model_name='porto_seguro_safe_driver_model.pkl')